In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import pyproj
import numbers
from pathlib import Path
from helpers import *

In [2]:
soup = BeautifulSoup(requests.get('https://www.waldbrandgefahr.ch/de/aktuelle-gefahrenlage').text, 'html.parser')

In [3]:
def project_coords(coords, from_proj, to_proj):
    if len(coords) < 1:
        return []

    if isinstance(coords[0], numbers.Number):
        from_x, from_y = coords
        to_x, to_y = pyproj.transform(from_proj, to_proj, from_x, from_y)
        return [to_x, to_y]

    new_coords = []
    for coord in coords:
        new_coords.append(project_coords(coord, from_proj, to_proj))
    return new_coords


def project_feature(feature, from_proj, to_proj):
    if not 'geometry' in feature or not 'coordinates' in feature['geometry']:
        print('Failed project feature', feature)
        return None

    new_coordinates = project_coords(feature['geometry']['coordinates'], from_proj, to_proj)
    feature['geometry']['coordinates'] = new_coordinates
    return feature


In [4]:
from this import d

div = soup.select_one('#fire_map_detail')
url = json.loads(div.attrs['data-react-props'])['geojson']
url = 'https://www.waldbrandgefahr.ch' + url

r = requests.get(url)
geojson = r.json()
#geojson = {"features": []}

# Sort
geojson['features'] = sorted(geojson['features'], key=lambda x: x['properties']['level'])

colors = {
    1: '#cfdd9d',
    2: '#e0d4b1',
    3: '#d47340',
    4: '#ce4731',
    5: '#93144b'
}

legende = {
    1: 'Keine oder geringe Gefahr',
    2: 'Mässige Gefahr',
    3: 'Erhebliche Gefahr',
    4: 'Grosse Gefahr',
    5: 'Sehr grosse Gefahr'
}

for feature in geojson['features']:
    del feature['bbox']
    del feature['id']
    feature['properties'] = {
        'fill': colors[feature['properties']['level']],
        'label': legende[feature['properties']['level']],
        'fill-opacity': 0.8,
        'stroke-opacity': 0
    }

mercator = pyproj.Proj(init='epsg:4326')
swissgrid = pyproj.Proj(init='epsg:2056')

projected_features = []
for feature in geojson['features']:
    projected_features.append(project_feature(feature, swissgrid, mercator))
geojson['features'] = projected_features


#json.dump(geojson, open('test.geojson', 'w'))

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


/Users/simon/Documents/projects/st-methods/bots/wetterereignisse_live_ch/env/lib/python3.9/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/simon/Documents/projects/st-methods/bots/wetterereignisse_live_ch/env/lib/python3.9/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/xv/8dpn0yrd2_sdyj5_kf48mxbh0000gn/T/ipykernel_40874/356897867.py:7: DeprecationWarn

In [18]:
labels = [
    {
        "coordinates": [8.540448763866461, 47.377069663836046],
        "type": "point",
        "label": "Zürich",
        "labelPosition": "left"
    },
    {
        "coordinates": [7.448853873224696, 46.95054213418904],
        "type": "point",
        "label": "Bern",
        "labelPosition": "left"
    },
    {
        "coordinates": [6.138221654376033, 46.2064440296011],
        "type": "point",
        "label": "Genf",
        "labelPosition": "right"
    },
    {
        "coordinates": [8.950669089563268, 46.01118348612857],
        "type": "point",
        "label": "Lugano",
        "labelPosition": "left"
    },
    {
        "coordinates": [9.53560206455592, 46.8509515603594],
        "type": "point",
        "label": "Chur",
        "labelPosition": "right"
    },
    {
        "coordinates": [9.375378684759603, 47.42547357986716],
        "type": "point",
        "label": "St. Gallen",
        "labelPosition": "right"
    },
    {
        "coordinates": [7.595953393405145, 47.561448902150964],
        "type": "point",
        "label": "Basel",
        "labelPosition": "left"
    },
    {
        "coordinates": [7.364787560405546, 46.23149747652774],
        "type": "point",
        "label": "Sitten",
        "labelPosition": "right"
    },
    {
        "coordinates": [8.308714014295267, 47.05150580099424],
        "type": "point",
        "label": "Luzern",
        "labelPosition": "bottom"
    },
]
# Städte umwandeln
city_features = list(map(lambda x: {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": x['coordinates']
            },
            "properties": {
                "type": x["type"],
                "label": x["label"],
                "labelPosition": x["labelPosition"],
            }
          }, labels))

# Open
background = json.load(open(Path('../data/background_ch.geojson'), 'r'))

features = [
    geojson,
    background,
] + city_features

update_chart(
    id = 'd0be298e35165ab925d72923355c5379',
    geojsonList = features,
)

Successfully updated item with id d0be298e35165ab925d72923355c5379 on production environment


In [12]:
test = pd.DataFrame()

In [15]:
test is None

False